In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from scipy.cluster import hierarchy

FLAG_fig = False

#### 富山県の市町村別人口動態
人口移動調査　http://www.pref.toyama.jp/sections/1015/lib/jinko/　の中にある

In [ ]:
url = 'http://www.pref.toyama.jp/sections/1015/lib/jinko/_dat_h29/jinko_dat005.xls'
data_orig = pd.read_excel(url, header=None)

データのフォーマットを見て，必要，不必要な行，列を判別する

In [ ]:
data_orig.head()

In [ ]:
data = data_orig.drop([0,1,2,3,4],axis=0)
# drop=True: indexの降り直し,  inplace=True: 書き換えは自身のメモリで行う
data.reset_index(drop=True, inplace=True)
data.head()

上記を見て，市の名前（カラム0），自然増加（カラム2），転入総数（カラム6），転出総数（カラム9）を用いることとし，これをDataFrame dfに結合する

In [ ]:
df = pd.concat([data.iloc[:,0], data.iloc[0:,2], data.iloc[0:,6], data.iloc[0:,9]], axis=1)
df.head()

dfの各列にラベルを与える, city:市の名前，natural:自然増加, in:転入総数，out:転出総数

In [ ]:
df.columns = ['city', 'natural', 'in', 'out']
df.head()

相関図（横軸：自然増加，縦軸：転入総数）をプロットする。このとき，各点に市の名前を付加する

In [ ]:
names = df.city
city_labels = list(names) # dendrogramへの入力はリスト形式

In [ ]:
fig = plt.subplots(figsize=(8,4)) # (22,18)
plt.rcParams['font.family'] ='Yu Mincho' # 日本語フォントを使うため
plt.xlabel('natural')
plt.ylabel('in')
plt.scatter(df['natural'], df['in'])
for i, text in enumerate(df.city):
    plt.annotate(text, xy=(df['natural'][i], df['in'][i]), size=10 )

if FLAG_fig: plt.savefig('fig_ClusterAggr_Toyama2.png')
plt.show()

df2 は，クラスタ分析のために，cityを削除したデータフレームである

In [ ]:
fig = plt.subplots(figsize=(8,4))
df2 = df.drop('city',axis=1)
Z = hierarchy.linkage(df2, 'ward')
dn = hierarchy.dendrogram(Z, labels=city_labels, leaf_rotation=90.)

#plt.tight_layout() # prevent the lack of label
if FLAG_fig: plt.savefig('fig_ClusterAggr_Toyama3.png', bbox_inches='tight')
plt.show()

#### 相関図
念のため，各変数同士の相関図を見る。<br>
seabornのプロットは，バージョン0.9.0から，この例のようなデータセットの種類の数に応じた色数を明示的に用意することが求められる。<br>
そうでないと，次のエラーが現れる "ValueError: color kwarg must have one color per dataset"<br>
ここでは，簡単に，hue='out'と指定した。ここに，'out'はdfのラベル名。幾つかのWarningが現れるが，気にしないことにする。

In [ ]:
sns.pairplot(df, hue='out')

if FLAG_fig: plt.savefig('fig_ClusterAggr_Toyama1.png')
plt.show()

上記の結果を見て，in, outに強い相関が有ることが認められる。naturalと（in, out）にも負の相関が認められるが，若干，外れる市町村があり，この原因究明も興味深い調査となるであろう。